In [1]:
import yahoo_fin as yf
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as si
import os
os.chdir('E:\StockStragegy\HistoricalStockPrice')
from collections import Counter

In [2]:
Dow_tickers=si.tickers_dow()
Nasdaq_tickers=si.tickers_nasdaq()
sp500_tickers=si.tickers_sp500()

In [18]:
print(len(Dow_tickers),len(Nasdaq_tickers),len(sp500_tickers))

30 4139 505


In [19]:
tickers=list(set(Dow_tickers+Nasdaq_tickers+sp500_tickers))
len(tickers)

4501

In [30]:
for ticker in not_tickers:
    try:
        prices=si.get_data(ticker)
        prices.to_excel(ticker+".xlsx")
    except:
        print(ticker)

PRN
ZCZZT
BRK.B
ZVZZC
ZXYZ.A
BF.B
ZBZZT
ZAZZT


In [31]:
exist_tickers=[x[:-5] for x in os.listdir()]

In [32]:
len(exist_tickers)

4494